In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from itertools import cycle
from statistics import mean 
import random
from random import randint
import numpy as np
import time
from datetime import date

In [2]:
symbols = []
companyNames = []
companySectors = []
userAgentList = []

useragents = open("useragents.txt", "r")

for line in useragents:
    userAgentList.append(line.replace('\n', ''))   

In [3]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [4]:
def getSymbolsCSV(fileName):
    
    global symbols
    
    print('\nLoading data from ' + fileName)

    with open(fileName) as csvfile:
    
        readCSV = csv.reader(csvfile, delimiter=',')
        
        for row in readCSV:
            
            symbol  = str(row[0]).replace('ï»¿', '')
            
            symbols.append(symbol)  
                
getSymbolsCSV('OptionsTradingStocks.csv')


Loading data from OptionsTradingStocks.csv


In [26]:
def getOptionsSpreads(sym):
    
    reload = True
    
    while reload == True:
        
        url = 'https://finance.yahoo.com/quote/' + sym + '/options?date=1613088000'

        agent = random.choice(userAgentList)
        headers = {'User-Agent': agent}

        page = requests.get(url, headers=headers, proxies = {"http": next(proxyPool)})
        soup = BeautifulSoup(page.text, 'html.parser')

        #print(soup)

        optionChain = soup.find(id='Col1-1-OptionContracts-Proxy')

        if optionChain != None:
            
            options = optionChain.find_all('tr', class_='in-the-money')
            reload = False
            calls = []

            for x in options:
                
                x = str(x)
                
                quote = x[x.find('href="/quote/') : x.find('href="/quote/') + 100]
                
                if 'P00' in quote:
                    break
                    
                else:
                    
                    strike = x[x.find('options?strike=') + 15 : x.find('options?strike=') + 22]
                    strike = strike.replace('&amp', '').replace('&a', '').replace(';', '').replace('m', '')

                    askPrice = x[x.find('options?strike=') + 240 : x.find('class="data-col6 ')]
                    askPrice = askPrice[askPrice.find('>') + 1 : askPrice.find('<')]
                    
                    print(sym, strike, askPrice)

                    calls.append([strike, askPrice])
            
            
            spreads = []
            
            
            for y in range(len(calls)):
                
                buyCallStrike = calls[y][0]
                buyCallAskPrice = calls[y][1]
                
                for z in range(y + 1, len(calls)):
                
                    sellCallStrike = calls[z][0]
                    sellCallAskPrice = calls[z][1]
                    
                
                    strikeDiff = float(sellCallStrike) - float(buyCallStrike)
                        
                    premium = round(((-1 * float(buyCallAskPrice)) + float(sellCallAskPrice)) * 100, 2)
                    
                    maxProfit = strikeDiff * 100 - abs(premium)
                    
                    percentReturn = round((maxProfit / abs(premium)) * 100, 2)
                    
                    spreads.append([buyCallStrike, buyCallAskPrice, sellCallStrike, sellCallAskPrice, strikeDiff, premium, maxProfit, percentReturn])
            
                    #print('(' + str(buyCallStrike) + ', ' + str(buyCallAskPrice) + ')', '(' + str(sellCallStrike) + ', ' + str(sellCallAskPrice) + ')', strikeDiff, premium, maxProfit)

            
            
            spreads = sorted(spreads,key=lambda l:l[-1], reverse=True)
            print('\n***', sym, '***\n')
            
            for s in spreads[0 : 20]:
                print('BTO', str(s[0]) + 'c', 'Cost: ' + str(s[1]))
                print('STO', str(s[2]) + 'c', 'Cost: ' + str(s[3]))
                print('Entry Cost:', s[5])
                print('Max Profit:', s[6], '+' + str(s[7]) + '%')
                print('----------------------------\n')
                

In [28]:
stock = input('Enter the stock you want to analyze: ')
getOptionsSpreads(stock)

Enter the stock you want to analyze: WMT
WMT 120 26.50
WMT 121 25.55
WMT 125 21.45
WMT 130 16.80
WMT 131 15.00
WMT 132 14.90
WMT 133 13.65
WMT 134 12.10
WMT 135 10.50
WMT 136 10.85
WMT 137 8.35
WMT 138 7.25
WMT 139 6.20
WMT 140 5.35
WMT 141 4.55
WMT 142 3.40
WMT 143 2.54
WMT 144 1.86
WMT 145 1.28
*** WMT ***

BTO 139c Cost: 6.20
STO 145c Cost: 1.28
Entry Cost: -492.0
Max Profit: 108.0
----------------------------

BTO 138c Cost: 7.25
STO 145c Cost: 1.28
Entry Cost: -597.0
Max Profit: 103.0
----------------------------

BTO 137c Cost: 8.35
STO 145c Cost: 1.28
Entry Cost: -707.0
Max Profit: 93.0
----------------------------

BTO 140c Cost: 5.35
STO 145c Cost: 1.28
Entry Cost: -407.0
Max Profit: 93.0
----------------------------

BTO 131c Cost: 15.00
STO 132c Cost: 14.90
Entry Cost: -10.0
Max Profit: 90.0
----------------------------

BTO 142c Cost: 3.40
STO 145c Cost: 1.28
Entry Cost: -212.0
Max Profit: 88.0
----------------------------

BTO 131c Cost: 15.00
STO 136c Cost: 10.85
Entry Co